In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [3]:
properties = pd.read_csv('/Users/cmarr/Documents/Travis_data_science/Kaggle/zillow/data/properties_2016.csv')

/Applications/anaconda/envs/zillow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train = pd.read_csv('/Users/cmarr/Documents/Travis_data_science/Kaggle/zillow/data/train_2016.csv', parse_dates = ["transactiondate"])

In [5]:
merged = pd.merge(train,properties,how='left',on='parcelid')
merged['transaction_month'] = merged['transactiondate'].dt.month
merged['lat_bins'] = pd.cut(merged.latitude, bins=100, labels=False)
merged['lon_bins'] = pd.cut(merged.longitude, bins=100, labels=False)

nan_subs = {'airconditioningtypeid': 0, 'architecturalstyletypeid': 0, 'basementsqft': 0,
           }

x_train = merged.drop(['parcelid', 'logerror','transactiondate'], axis=1)
target = merged['logerror']

print(x_train.head())
print(target.head())

   airconditioningtypeid  architecturalstyletypeid  basementsqft  bathroomcnt  \
0                    1.0                       NaN           NaN          2.0   
1                    NaN                       NaN           NaN          3.5   
2                    1.0                       NaN           NaN          3.0   
3                    1.0                       NaN           NaN          2.0   
4                    NaN                       NaN           NaN          2.5   

   bedroomcnt  buildingclasstypeid  buildingqualitytypeid  calculatedbathnbr  \
0         3.0                  NaN                    4.0                2.0   
1         4.0                  NaN                    NaN                3.5   
2         2.0                  NaN                    4.0                3.0   
3         2.0                  NaN                    4.0                2.0   
4         4.0                  NaN                    NaN                2.5   

   decktypeid  finishedfloor1squ

In [42]:
'''
Need to:
- Drop useless variables like census tract (at least for now)
- Create indicators of missingness
- Fill in NA's with unique values
- 
'''
merged.bathroomcnt.value_counts(dropna=False)

 2.0     36534
 3.0     19946
 1.0     15035
 2.5      8091
 4.0      3743
 1.5      1641
 0.0      1165
 5.0      1142
 3.5      1091
 4.5       795
NaN        536
 6.0       448
 5.5       253
 7.0       155
 8.0       114
 6.5        54
 9.0        36
 10.0       14
 7.5         9
 11.0        3
 12.0        3
 20.0        1
 8.5         1
 15.0        1
Name: bathroomcnt, dtype: int64

In [49]:
test = merged[merged['bathroomcnt'].isnull()]

In [51]:
test[['parcelid', 'latitude', 'longitude', 'airconditioningtypeid', 'transactiondate']].head(5)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transaction_month,lat_bins,lon_bins
840,14285150,-0.4080,2016-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
937,13951364,-0.2332,2016-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1389,14356139,0.0573,2016-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2148,14673130,0.0602,2016-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2272,14356133,-0.0387,2016-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [38]:
merged[merged['parcelid']==14650661]

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transaction_month,lat_bins,lon_bins
39255,14650661,0.0917,2016-05-26,3.0,NaN,NaN,1.5,1.0,NaN,NaN,1.5,NaN,NaN,837.0,837.0,NaN,NaN,NaN,NaN,6059.0,NaN,1.0,1.0,0.0,NaN,13.0,33609400.0,-117748000.0,NaN,1.0,NaN,NaN,NaN,1.0,34,266.0,NaN,6.059063e+07,46098.0,1286.0,NaN,96963.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,1973.0,1.0,NaN,74764.0,183250.0,2015.0,108486.0,1931.74,NaN,NaN,6.059063e+13,5,18.0,89.0


In [15]:
parcels = properties.parcelid.unique()
test = pd.DataFrame(parcels, columns = ['ParcelId'])
test['201610'] = 0
test['201611'] = 0
test['201612'] = 0
test['201710'] = 0
test['201711'] = 0
test['201712'] = 0
test.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [16]:
test.to_csv('data/submission.csv', index=False)

In [17]:
from sklearn.ensemble import GradientBoostingRegressor as GBR

In [126]:
params = {'n_estimators': 50, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'lad'}
model = GBR(**params)
model.fit(X=x_train, y=target)

ValueError: could not convert string to float: 'Y'

In [67]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [112]:
# create model
lamap = merged.copy(deep=True)
lamap.dropna(axis=0, how='any', inplace=True)
X = lamap[['lat_bins', 'lon_bins']]
months = pd.get_dummies(lamap.transaction_month)
x = X.join(months)
y = lamap[['logerror']]
# x.isnull().groupby(['latitude', 'longitude']).count()
X = x.as_matrix()
Y = y.as_matrix()

In [113]:
model = Sequential()
model.add(Dense(60, input_dim=14, activation='sigmoid'))
model.add(Dense(60, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [124]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=500, batch_size=100)

Epoch 1/500
90275/90275 [==============================] - 4s - loss: 0.0680 - acc: 0.0094     
Epoch 2/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 3/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 4/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 5/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 6/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 7/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 8/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 9/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 10/500
90275/90275 [==============================] - 3s - loss: 0.0680 - acc: 0.0094     
Epoch 11/500
90275/90275 [=============

90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 168/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 169/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 170/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 171/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 172/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 173/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 174/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 175/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 176/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 177/500
90275/90275 [=======

90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 334/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 335/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 336/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 337/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 338/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 339/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 340/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 341/500
90275/90275 [==============================] - 3s - loss: 0.0679 - acc: 0.0094     
Epoch 342/500
90275/90275 [==============================] - 2s - loss: 0.0679 - acc: 0.0094     
Epoch 343/500
90275/90275 [=======

90275/90275 [==============================] - 2s - loss: 0.0678 - acc: 0.0094     
Epoch 500/500
90275/90275 [==============================] - 2s - loss: 0.0678 - acc: 0.0094     


In [99]:
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

90275/90275 [==============================] - 3s     

acc: 0.94%
